# Final Case Study


#### Steven Hayden, Kevin Mendonsa, Joe Schueder, Nicole Wittlin  

## Executive Summary Framework

Intro?
Business Imperative?
Put those here??


Key Findings About Data Set/Observations 
- Info about 
- Wed most observations 100K, Tues & Thurs approx 30K
- July most observations, close second in June; August and May close. Dec, Jan, Feb minimal. possible seaonal pattern
- Asia 139K observations, 
- x32 a continuous varible as a percentage but low cardinality with 11 unique values, opted to treat as categorical rather than continuous
- Distribution 

Baseline Model 
- Log Reg, what were the criteria, results

Other Models Run

Summary Chart of Costs from Excel

Conclusion 


=====

---
## Introduction 

We have been provided a dataset comprising 50 explanatory variables and one response variable with the objective of minimizing cost of the predicted response using provided cost penalties to the business. The dataset contains 160,000 observations. Given the lack of domain knowledge and intrinsic information about the dataset, we will conduct a comprehensive exploratory data analysis (EDA) to understand the charactersitics of the data and the variables. The response variable "y" is binary (1,0). 

No additional information has been provided and therefore the data analysis and processing of the information is critical for delivering optimal results.  We will clearly articulate any assumptions we make in order to reach our conclusions.

---

## Business Imperative
The business imperative for this analysis is to reduce False Negatives as a priority as they appear to have a greater adverse impact to the business in question.  False Positives while important are 50 times less impactful than False Negatives.

We have therefore been informed that each False Positive prediction will cost the business \\$10 and each False Negative prediction will cost the business \\$500. True Positives and True Negatives have no effect on the business cost. Given that False Negatives have a higher cost penalty of (\\$500), Recall scores are critical in this analysis. Precision is also important with a cost penalty of \\$10. Correct predictions are "zero cost". 

The delivered model must be able to be deployed in a production environment and be generalized for future application of new data.  We will methodically step through various steps from cursory data review to an in-dept data analysis, baselining, modeling, dimensionality reduction, optimization and tuning and conclusions.

---

## Methodology and organization of our analysis
Our analysis will step through the following stages building upon the conclusions and decisions of previous findings as we transition to an optimal conclusion to support the business imperative for this analysis.

### Exploratory Data Analysis (EDA)
- Data classes of features
- Validate Normality assumptions
- Check Cardinality
- Missing Data
- Validate Independence assumptions
    - Pearson's Correlation
    - Phi K Correlation
- Outliers


### Preprocessing
- **Data Cleanup**
    - Renaming values
    - Stripping out special characters
    - Conversion of data classes - String to float
  

- **Imputation Strategy to address missing data**
    - Impute or drop
    
    
- **One-hot Encoding of categorical variables**


- **Standardization**
    - MinMax scaling
    
    
- **Balancing**
    
    
- **Sampling strategy for testing and training splits**


### Establish a Baseline
- Modeling using diverse machine learning model algorithms (rationale for models selected)
- Develop a baseline Cost-Benefit Matrix based on provided cost penalties
 
###  Dimensionality Reduction (Feature Reduction)
- Recursive feature extraction (Dimensionality Reduction - Occams's Razor)
 
###  Cross Validation using K-fold (Model generalization and optimization)
 
###  Parameter Tuning and Optimization
- RandomizedSearchCV to choose best parameters

### Modeling with Reduced Features
 
###  Ensembling
- Method using multiple learning algorithms to derive improved predictive performance than a single algorithm.
 
###  Error Metrics
- ROC-AUC
- Recall
- Precision
- F1 scores
- Confusion Matrix - Model accuracy and predictive power

###   Conclusion and findings
- Cost/Benefits Matrix - Translate into business terms/discovery
---

In [1]:
# Import the necessary packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier
import itertools
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.feature_selection import RFECV
from vecstack import stacking
from mlxtend.classifier import StackingCVClassifier
from mlxtend.plotting import plot_learning_curves, plot_decision_regions
from sklearn.pipeline import Pipeline
#from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors
from sklearn.calibration import CalibratedClassifierCV
from pandas_profiling import ProfileReport

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter('ignore')

# Exploratory Data Analysis

### Load the data to the workspace

- Load the data as a data frame and conduct cursory analysis to review
    - Shape (rows and columns)
    - Variable Classes (datatypes)
    - Missingness of data (NULL data)
    - Unique values for Cardinality

For clarity and consistency we will refer to Explanatory variables as Features.

In [2]:
# Load the data
%time Business_Data = pd.read_csv('final_project.csv')

Wall time: 1.57 s


In [3]:
# Explore the data
Business_Data.shape

(160000, 51)

In [4]:
# Check data type and not null counts of all the columns
Business_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 51 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   x0      159974 non-null  float64
 1   x1      159975 non-null  float64
 2   x2      159962 non-null  float64
 3   x3      159963 non-null  float64
 4   x4      159974 non-null  float64
 5   x5      159963 non-null  float64
 6   x6      159974 non-null  float64
 7   x7      159973 non-null  float64
 8   x8      159979 non-null  float64
 9   x9      159970 non-null  float64
 10  x10     159957 non-null  float64
 11  x11     159970 non-null  float64
 12  x12     159964 non-null  float64
 13  x13     159969 non-null  float64
 14  x14     159966 non-null  float64
 15  x15     159965 non-null  float64
 16  x16     159974 non-null  float64
 17  x17     159973 non-null  float64
 18  x18     159960 non-null  float64
 19  x19     159965 non-null  float64
 20  x20     159962 non-null  float64
 21  x21     15

There are 5 categorical features and 45 continuous features of float64 data type.  The response variable is binary and classed as an integer.

In [5]:
# Use Pandas Profiling to do a quick EDA 
# (Executing Mini version due to performance and memory constraints)
from pandas_profiling import ProfileReport
profileMin = ProfileReport(Business_Data, minimal=True)
profileMin.to_notebook_iframe()
# profileMin.to_file(output_file="output_min.html") # save profiling report to file

---
### Normality Assumption
In reviewing the results of the Pandas profiling report above, we can see that the continuous features meet the assumptions for normality with well defined Gaussian distributions.

---
### Missing data (NULLs) - Analysis by feature for determining imputation strategy
---

In [6]:
# Get NULL counts by column
Business_Data.isnull().sum()

x0     26
x1     25
x2     38
x3     37
x4     26
x5     37
x6     26
x7     27
x8     21
x9     30
x10    43
x11    30
x12    36
x13    31
x14    34
x15    35
x16    26
x17    27
x18    40
x19    35
x20    38
x21    29
x22    27
x23    47
x24    28
x25    22
x26    36
x27    30
x28    35
x29    30
x30    30
x31    39
x32    31
x33    41
x34    41
x35    30
x36    27
x37    23
x38    31
x39    23
x40    36
x41    40
x42    26
x43    37
x44    40
x45    29
x46    31
x47    37
x48    32
x49    32
y       0
dtype: int64

---
- TOTAL NULL counts of all features: **1608**
- Percentage of NULL values against total data: 1608/160000 => **1.005%**

#### The Imputation strategy and actions are listed further below

---
### Inspect all features for Cardinality (Unique classes per variable)
This will enable the identification of cardinality in features and also identify candidates for conversion to a categorical data class.

In [7]:
# Examine Unique values by feature
Business_Data.nunique(dropna=False)

x0     159975
x1     159976
x2     159963
x3     159964
x4     159975
x5     159964
x6     159975
x7     159974
x8     159980
x9     159971
x10    159958
x11    159971
x12    159965
x13    159970
x14    159967
x15    159966
x16    159975
x17    159974
x18    159961
x19    159966
x20    159963
x21    159972
x22    159974
x23    159954
x24         4
x25    159979
x26    159965
x27    159971
x28    159966
x29        13
x30         6
x31    159962
x32        13
x33    159960
x34    159960
x35    159971
x36    159974
x37    129199
x38    159970
x39    159978
x40    159965
x41    159961
x42    159975
x43    159964
x44    159961
x45    159972
x46    159970
x47    159964
x48    159969
x49    159969
y           2
dtype: int64

### Closer review of the categorical features identified above
- A closer analysis reveals that features **x32** and **x37** have special characters. 
- Additionally, feature **x32** has **low cardinality** with only 13 unique classes in 160,000 observations. 

In [8]:
# Review and analyze the Categorical features and response as indicated above
print('Unique classes of feature x24:: \n',Business_Data.x24.value_counts(dropna = False))
print('\nUnique values of feature x29:: \n',Business_Data.x29.value_counts(dropna = False))
print('\nUnique values of feature x30:: \n',Business_Data.x30.value_counts(dropna = False))
print('\nUnique values of feature x32:: \n',Business_Data.x32.value_counts(dropna = False))
print('\nUnique values of response y:: \n',Business_Data.y.value_counts(dropna = False))

Business_Data.x37.head()

Unique classes of feature x24:: 
 asia       138965
euorpe      16538
america      4469
NaN            28
Name: x24, dtype: int64

Unique values of feature x29:: 
 July       45569
Jun        41329
Aug        29406
May        21939
sept.      10819
Apr         6761
Oct         2407
Mar         1231
Nov          337
Feb          140
NaN           30
Dev           23
January        9
Name: x29, dtype: int64

Unique values of feature x30:: 
 wednesday    101535
thurday       29429
tuesday       27954
friday          564
monday          488
NaN              30
Name: x30, dtype: int64

Unique values of feature x32:: 
 0.01%     40767
-0.01%    34094
0.0%      33923
-0.0%     30492
-0.02%     9924
0.02%      7987
-0.03%     1727
0.03%       855
-0.04%      138
0.04%        55
NaN          31
-0.05%        6
0.05%         1
Name: x32, dtype: int64

Unique values of response y:: 
 0    95803
1    64197
Name: y, dtype: int64


0     $1313.96
1     $1962.78
2      $430.47
3    $-2366.29
4     $-620.66
Name: x37, dtype: object

# Correlation

We used Pearson's correlation to ensure the assumptions of independence of features are met. The correlation plot identifies attributes in the original dataset that are highly collinear. In other words, one predictor feature in the regression model can be linearly predicted from the others with a substantial degree of accuracy.


### Pearson's Correlation

The Pearson correlation coefficient is used to measure the strength of a linear association between two variables, where the value r = 1 means a perfect positive correlation and the value r = -1 means a perfect negataive correlation. It inofrms whether a statistically significant linear relationship exists between two continuous variables. The strength of a linear relationship (i.e., how close the relationship is to being a perfectly straight line) 

(Reference: https://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/)

Interpretation of Pearson's coefficient.
* Exactly –1. A perfect downhill (negative) linear relationship

* –0.70. A strong downhill (negative) linear relationship

* –0.50. A moderate downhill (negative) relationship

* –0.30. A weak downhill (negative) linear relationship

* 0. No linear relationship

* +0.30. A weak uphill (positive) linear relationship

* +0.50. A moderate uphill (positive) relationship

* +0.70. A strong uphill (positive) linear relationship

* Exactly +1. A perfect uphill (positive) linear relationship

![title](img/PearsonCorrelation2.png) 

Based on the analysis of the correlation matrix above, features $x6$ and $x41$ were excluded as they were highly correlated with $x2$ and $x38$ respectively with a correleation value = 1. Therefore, we will drop these variables from our analysis.

In [9]:
# Drop features 'x6', 'x41' due to high correlation as identified above
Project_Data_final = Business_Data.drop(['x6', 'x41'], axis = 1)

# Pre-Processing

### Clean-up of categorical features - x32 and x37
As identified above, we need to clean feature x37 by stripping it off special characters and converting it to a class "float".

#### Actions:
- Strip feature **x37** off all special characters 
- Convert feature **x37** to a float data class
- Retain feature **x32** as a categorical feature

In [10]:
# Strip special characters in feature x37
# Convert feature x37 to float data class
Business_Data['x37'] = Business_Data['x37'].str.replace('$','').astype(float)

In [11]:
Business_Data.x32.head()

0      0.0%
1    -0.02%
2    -0.01%
3     0.01%
4     0.01%
Name: x32, dtype: object

In [12]:
# Feature x32 class distribution
# Business_Data['x32'] = Business_Data['x32'].astype(str)
Business_Data.x32.value_counts()

0.01%     40767
-0.01%    34094
0.0%      33923
-0.0%     30492
-0.02%     9924
0.02%      7987
-0.03%     1727
0.03%       855
-0.04%      138
0.04%        55
-0.05%        6
0.05%         1
Name: x32, dtype: int64

In [13]:
Business_Data.x37.head()

0    1313.96
1    1962.78
2     430.47
3   -2366.29
4    -620.66
Name: x37, dtype: float64

# Imputation strategy

Impute the nulls with appropriate values after determining if they are 
- Missing Completely At Random (MCAR)
- Missing At Random (MAR)
- Missing Not At Random (MNAR)

After discussions with the business, it has been determined that the missing data is to be treated as Missing At Random (MAR).  Furthermore, since almost all the continuous features have a gaussian distribution and meet the assumptions for normality (refer to Pandas Profiling report for additional details), it was agreed to replace NULL values in continuous features with the "median" of the respective feature.  For missing values or NANs in categorical values we will proceed with dropping the records as they represent just **0.07% of the 160,000** records. 

#### Analysis:
- TOTAL NULL counts of all features: **1608**
- Percentage of total data: 1608/160000 => **1.005%**

#### Actions:
- Impute missing values of continuous features with the **median**
- Drop missing values for categorical features given the large dataset of 160000 records


In [14]:
# Create a new data frame and replace null values as 
# per the imputation strategy agreed with the business.
# Replace missing values in continuous features using the median
Business_Data_imputed = Business_Data.apply(lambda x: x if x.dtype == 'object' else x.fillna(x.median()))

In [15]:
Business_Data_imputed.isnull().sum()

x0      0
x1      0
x2      0
x3      0
x4      0
x5      0
x6      0
x7      0
x8      0
x9      0
x10     0
x11     0
x12     0
x13     0
x14     0
x15     0
x16     0
x17     0
x18     0
x19     0
x20     0
x21     0
x22     0
x23     0
x24    28
x25     0
x26     0
x27     0
x28     0
x29    30
x30    30
x31     0
x32    31
x33     0
x34     0
x35     0
x36     0
x37     0
x38     0
x39     0
x40     0
x41     0
x42     0
x43     0
x44     0
x45     0
x46     0
x47     0
x48     0
x49     0
y       0
dtype: int64

#### Drop NULL observations in categorical features

Given the extremely low count of NULLs in the categorical features, dropping these observations still retain 159912 of the 160000 rows. This is approximately **0.07%** of the original data set.  It should likely have little to no impact on the overall analysis.

- x24 - $28$
- x29 - $30$
- x30 - $30$
- x32 - $31$

In [16]:
# Drop NULLS in categorical features
Business_Data_imputed.dropna(inplace=True)

# Review the data post dropping the NaNs in categorical features
Business_Data_imputed.isnull().sum()

x0     0
x1     0
x2     0
x3     0
x4     0
x5     0
x6     0
x7     0
x8     0
x9     0
x10    0
x11    0
x12    0
x13    0
x14    0
x15    0
x16    0
x17    0
x18    0
x19    0
x20    0
x21    0
x22    0
x23    0
x24    0
x25    0
x26    0
x27    0
x28    0
x29    0
x30    0
x31    0
x32    0
x33    0
x34    0
x35    0
x36    0
x37    0
x38    0
x39    0
x40    0
x41    0
x42    0
x43    0
x44    0
x45    0
x46    0
x47    0
x48    0
x49    0
y      0
dtype: int64

In [17]:
# Clean-up classes in some categorical features for consistency and spelling errors
Business_Data_imputed['x24'] = Business_Data_imputed['x24'].replace(['asia', 'euorpe','america'], ['Asia', 'Europe','America'])
Business_Data_imputed['x29'] = Business_Data_imputed['x29'].replace(['sept.', 'January','Dev','July'], ['Sep', 'Jan','Dec','Jul'])
Business_Data_imputed['x30'] = Business_Data_imputed['x30'].replace(['monday', 'tuesday','wednesday','thurday','friday'], ['Mon', 'Tue','Wed','Thu','Fri'])

In [18]:
# Review unique classes and counts for categorical variables
print('Unique values of Business_Data x24:: \n',Business_Data_imputed.x24.value_counts(dropna = False))
print('\nUnique values of Business_Data x29:: \n',Business_Data_imputed.x29.value_counts(dropna = False))
print('\nUnique values of Business_Data x30:: \n',Business_Data_imputed.x30.value_counts(dropna = False))
print('\nUnique values of Business_Data x32:: \n',Business_Data_imputed.x32.value_counts(dropna = False))

Unique values of Business_Data x24:: 
 Asia       138880
Europe      16534
America      4467
Name: x24, dtype: int64

Unique values of Business_Data x29:: 
 Jul    45546
Jun    41299
Aug    29385
May    21932
Sep    10815
Apr     6758
Oct     2407
Mar     1231
Nov      336
Feb      140
Dec       23
Jan        9
Name: x29, dtype: int64

Unique values of Business_Data x30:: 
 Wed    101473
Thu     29413
Tue     27943
Fri       564
Mon       488
Name: x30, dtype: int64

Unique values of Business_Data x32:: 
 0.01%     40746
-0.01%    34078
0.0%      33902
-0.0%     30479
-0.02%     9916
0.02%      7981
-0.03%     1726
0.03%       854
-0.04%      137
0.04%        55
-0.05%        6
0.05%         1
Name: x32, dtype: int64


# One Hot encoding for Categorical Features

One Hot Encoding (ohe) is a popular approach in which a categorical feature is converted to a format that enhances the predictive capability of ML algorithms. It works best when the cardinality of the categorical feature is low but is not advisable for features where there are more than 15 different classes. 


![title](img/OneHotEncoding2.png)

Reference: https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding

#### Parameter options used for pd.get_dummies
- data - Data of which to get dummy indicators 
    - Parameter setting: **Project_Data_imputed**
- prefix_sep - If appending prefix, separator/delimiter to use. Or pass a list or dictionary as with prefix.
    - Parameter setting: **"_"**
- drop_first - Whether to get k-1 dummies out of k categorical levels by removing the first level
    - Parameter setting: **True**
- columns - Names as list of Categorical features for one hot encoding.
    - Parameter setting: **cat_columns** 

In [19]:
# Create a variable "cat_columns" to include all features to be encoded
cat_columns = ['x24', 'x29', 'x30','x32']

# Use the pandas get_dummies function to encode the selected categorical features
Business_Data_ohe = pd.get_dummies(Business_Data_imputed, prefix_sep="_", drop_first=True, columns=cat_columns)

In [20]:
# Review the results of the encoding operation
Business_Data_ohe.isnull().sum()
Business_Data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159881 entries, 0 to 159999
Data columns (total 75 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   x0          159881 non-null  float64
 1   x1          159881 non-null  float64
 2   x2          159881 non-null  float64
 3   x3          159881 non-null  float64
 4   x4          159881 non-null  float64
 5   x5          159881 non-null  float64
 6   x6          159881 non-null  float64
 7   x7          159881 non-null  float64
 8   x8          159881 non-null  float64
 9   x9          159881 non-null  float64
 10  x10         159881 non-null  float64
 11  x11         159881 non-null  float64
 12  x12         159881 non-null  float64
 13  x13         159881 non-null  float64
 14  x14         159881 non-null  float64
 15  x15         159881 non-null  float64
 16  x16         159881 non-null  float64
 17  x17         159881 non-null  float64
 18  x18         159881 non-null  float64
 19  x1

---
# Outliers

In [38]:
#boxplot of all the variables
#Business_Data_BoxPlot1 = Business_Data_ohe[['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9',]]
#Business_Data_BoxPlot2 = Business_Data_ohe[['x10','x11','x12','x13','x14','x15','x16','x17','x18','x19']]
#Business_Data_BoxPlot3 = Business_Data_ohe[['x20','x21','x22','x23','x24','x25','x26','x27','x28','x29']]
#Business_Data_BoxPlot4 = Business_Data_ohe[['x30','x31','x32','x33','x34','x35','x36','x37','x38','x39']]
#Business_Data_BoxPlot5 = Business_Data_ohe[['x40','x41','x42','x43','x44','x45','x46','x47','x48','x49']]
      
#boxplot of all the variables
plt.figure(figsize=(15, 15))
ax = drop_nan_df.boxplot()
ax.set_xticklabels(ax.get_xticklabels(), rotation=65, horizontalalignment='right')

---
# Standardization - Scaling of the data

A pre-processing step applied to independent features in a dataset. Primarily it normalizes the data within a particular range. It may also aid in accelerating the computations of an algorithm. The Features have been scaled to a mean of 0 and variance of 1 to improve the accuracy of the classification models.

- *fit_transform* within MinMaxScaler() function fits the data and then transforms it.
---

In [39]:
# Assign the MinMaxScaler to the variable scaler
scaler = MinMaxScaler()

# Perform the scaling using the scaler
Business_Data_ohe_scaled = pd.DataFrame(scaler.fit_transform(Business_Data_ohe), columns=Business_Data_ohe.columns)

# Review the data post scaling
Business_Data_ohe_scaled.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x32_-0.02%,x32_-0.03%,x32_-0.04%,x32_-0.05%,x32_0.0%,x32_0.01%,x32_0.02%,x32_0.03%,x32_0.04%,x32_0.05%
count,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,...,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000,159881.000000
mean,0.498395,0.484268,0.473770,0.476629,0.520278,0.487751,0.473779,0.525666,0.490752,0.505541,...,0.062021,0.010796,0.000857,0.000038,0.212045,0.254852,0.049918,0.005341,0.000344,0.000006
std,0.116206,0.116829,0.107960,0.108415,0.116644,0.110552,0.107965,0.092364,0.115988,0.114781,...,0.241195,0.103340,0.029260,0.006126,0.408758,0.435779,0.217777,0.072890,0.018544,0.002501
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.419959,0.405735,0.400452,0.403659,0.441475,0.413356,0.400452,0.466303,0.412554,0.428507,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.498073,0.484334,0.472208,0.476525,0.520300,0.487755,0.472214,0.527888,0.490927,0.505554,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.576522,0.563162,0.547144,0.550157,0.598995,0.562359,0.547160,0.585873,0.568981,0.583199,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


---
# Balancing

Looking at the distribution of the outcomes (response variable) in our dataset, we can see that almost 60% of the data in our set has a response of "0" while the outcome 1 makes up 40%. When we run our classifier, we will need to be careful to account for the slightly unbalanced nature of our classifiers to help ensure we aren't overclassifying the majority labels.

![title](img/ResponseVariable.png)


---
# Sampling
---

### Train - Test Split

This technique provides us a way of voluntarily holding back part of the data to test whether the model works. If we use our entire dataset to train the model, then the model will alway predict the correct category for any entry in the data set. This process of setting aside a part of the dataset voluntarily to evaluate the model stops the model from being too optimistic when predicting the outcome. Training the model on the entire data set could also lead to data snooping bias. This kind of bias results from refining too many parameters to improve the model's performance on a specific data set. In our analysis, we will split the data into training and test data sets (75:25) randomly. 

Reference: https://towardsdatascience.com/3-things-you-need-to-know-before-you-train-test-split-869dfabb7e50

In [40]:
# Assign the Explanatory features and Response variable to variables X and y respectively  
X = Business_Data_ohe_scaled.drop(columns=['y'], axis=1)
y = Business_Data_ohe_scaled.y

In [41]:
# We split the data but use the stratify option to ensure a 
# balanced split of the outcome "y" across the test and train data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10, stratify=y)

### Check the ratio of the response variable in the Train-Test split

Let's check the ratio of the response variable and compare it with that of the Train and Test dataset. The below results confirm that the ratio is retained and hence we can proceed with K-Fold Cross-Validation(CV) instead of Stratifid K-Fold CV.

In [42]:
sum(Business_Data_ohe_scaled['y']==0)/sum(Business_Data_ohe_scaled['y']==1)

1.4925324270391618

In [43]:
sum(y_test==0)/sum(y_test==1)

1.4925791968071838

In [44]:
sum(y_train==0)/sum(y_train==1)

1.4925168371164879

---
# Baseline Modeling
---


### Establish the baseline model and Cost-Benefit matrix before tuning and optimizations

- Use the complete data set and select diverse models with default parameters.
- Establish a confusion matrix with classification results and model accuracy scores. 
- Create a custom function **confusion_mat** to capture, display and compare the confusion and classification results. 
- Use the true False Positives(FP) and False Negatives(FN) for the case counts to calculate the resulting business cost.

---

###  MODEL EVALUATION METRICS 

In order to evaluate the model, we reviewed several standard metrics, which are defined below. The aim is to maximize our precision, recall, and accuracy scores in our models.


The model had the following evaluation profile calcualated based on averages of the metrics by evaluating them on 10 times cross validation:  
 * Accuracy: 
 * Precsion: 
 * Recall: 
 * F1:   


* **Confusion Matrix** - The confusion matrix shows that this model categorized XX incorrectly as 1 when they were not and xx emails as 0 when they actually were 1.  


  
* **ACCURACY** - total number of correct predictions (True Positives/TP; True Negatives/TN) over total number of predictions made. <br>
Accuracy = (TP + TN)/(TP + FP + FN + TN)


* **PRECISION** - proportion of true positives over total number of positive outcomes, whether accurately predicted (TP) or inaccurately predicted (FP); helps illuminate which model is accurately picking correct classes or correctly classifying observations.<br>
Precision = (TP) / (TP + FP)


* **RECALL** - proportion of positive outcomes that were correctly classified by model; tells how many values were incorrectly predicted; a good pair with precision to determine if modeling is overfitting or selecting a single class; also known as sensitivity.  
Recall/Sensitivity = (TP) / (TP + FN) 


* **F1 SCORE** - measure of accuracy that accounts for true negatives and false positives.<br>
F1 score = 2(True Positive Rate * True Negatives)/(True Positives + True Negatives) 


We further plotted an **ROC curve** to visualize the Decision Tree's performance. An ROC curve, also known as a Reciever Operation Characteristic Curve, plots and compares classifiers based on the True Positive Rates (TPR) and False Positive Rates (FPR) for each classifier. An AUC (Area Under the Curve) score of 1.0 denotes a perfect classifier and an area of 0.5 represents a model which is no better than a random guess. Higher the AUC the better the classifier.

---

In [45]:
# Establish the default parameters for the models to be used below
rand_state = 101 # set a seed for reproducibility
n_iterations = 5

---
### K-fold Cross-Validation

- This statistical method is used to evaluate model generalization performance for future applicability
- Provides greater stability and is more robust than using a simple split of a dataset into train and test sets.
- The algorithm splits the dataset multiple times training the model repeatedly on each of the splits.
- KFold divides the dataset into groups of equal sized samples (if possible), called folds.
- The prediction function uses these folds for learning, with the fold left out used for testing.

![title](img/KFoldCV.png) 

Reference: https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538

We adopted the k-fold cross-validation approach for our analysis with the following parameters.

Parameters for k-fold CV:
- **n_splitsint** - Number of folds
    - Parameter selected: **5**
- **random_state** - random_state is the seed used by the random number generator
    - Parameter selected: **10**

In [46]:
# Establish the cross validation parameters to be used in the models
cv = KFold(n_splits=n_iterations, random_state=rand_state)

# Provide description/definition of accuracy metrics being used below

In [47]:
# Create a dataframe to capture the results of the baseline models below
results_df = pd.DataFrame(columns=['Classifier', 'Features', 'Accuracy', 'Parameters', 'Recall', 
                                   'Precision', 'FPR', 'TPR', 'AUC', 'True_Positives', 
                                   'True_Negatives', 'False_Positives', 'False_Negatives', 'Cost(1000s)'])
# Review the dataframe
results_df

,Classifier,Features,Accuracy,Parameters,Recall,Precision,FPR,TPR,AUC,True_Positives,True_Negatives,False_Positives,False_Negatives,Cost(1000s)


In [48]:
# Create a custom function to create a confusion matrix and print the results of the base models
def confusion_mat(y_test, y_pred):
    print('----------------------------------------------------------')
    print('Confusion Matrix::\n')
    print(pd.crosstab(y_test, y_pred))
    print()
    print('Classification Report::')
    print(classification_report(y_test, y_pred))
    TN = pd.crosstab(y_test, y_pred).iloc[0, 0]
    FP = pd.crosstab(y_test, y_pred).iloc[0, 1]
    FN = pd.crosstab(y_test, y_pred).iloc[1, 0]
    TP = pd.crosstab(y_test, y_pred).iloc[1, 1]
    Cost = ((FP*10)+(FN*500))/1000
    Recall = recall_score(y_test, y_pred, average='weighted')
    Precision = precision_score(y_test, y_pred)
    Accuracy = accuracy_score(y_test, y_pred)
    return(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision)

In [49]:
# Create a classifier dictionary
clfsDict = {'Logistic Regression': LogisticRegression(solver = 'lbfgs', random_state=rand_state),
            'Gaussian Naive-Bayes': GaussianNB(),
            'Decision Tree': DecisionTreeClassifier(random_state=rand_state),
            'Random Forest': RandomForestClassifier(n_estimators=100, random_state=rand_state),
            'XGB': XGBClassifier(random_state=rand_state)}

In [50]:
start = time()

model_run_stats = []

for mdl, clf in clfsDict.items():
    start_1 = time()
    acc = cross_val_score(clf,X,y=y,cv=cv)
    clf.fit(X_train, y_train)
    y_pred_full_data = clf.predict(X_test)
    y_pred_proba_full_data = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test,  y_pred_proba_full_data)
    auc = roc_auc_score(y_test, y_pred_proba_full_data)
    (FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_full_data)
    results_df = results_df.append({'Classifier':mdl
                                                    , 'Features':'Full'
                                                    , 'Parameters': 'Default'
                                                    , 'Accuracy': Accuracy
                                                    , 'Recall': Recall
                                                    , 'Precision': Precision
                                                    , 'FPR': fpr
                                                    , 'TPR': tpr
                                                    , 'AUC': auc
                                                    , 'True_Positives': TP
                                                    , 'True_Negatives': TN
                                                    , 'False_Positives': FP
                                                    , 'False_Negatives': FN
                                                    , 'Cost(1000s)': Cost
                                                   }
                                                   , ignore_index=True
                                                  )
    print()
    print('Process Time:: %0f' %(time()-start_1))
    print()
    
model_run_stats

print('Wall Time:: %0f' %(time()-start))

----------------------------------------------------------
Confusion Matrix::

col_0    0.0   1.0
y                 
0.0    20019  3916
1.0     7786  8250

Classification Report::
              precision    recall  f1-score   support

         0.0       0.72      0.84      0.77     23935
         1.0       0.68      0.51      0.59     16036

    accuracy                           0.71     39971
   macro avg       0.70      0.68      0.68     39971
weighted avg       0.70      0.71      0.70     39971


Process Time:: 11.108629

----------------------------------------------------------
Confusion Matrix::

col_0    0.0   1.0
y                 
0.0    19249  4686
1.0     7729  8307

Classification Report::
              precision    recall  f1-score   support

         0.0       0.71      0.80      0.76     23935
         1.0       0.64      0.52      0.57     16036

    accuracy                           0.69     39971
   macro avg       0.68      0.66      0.66     39971
weighted avg  

In [53]:
# Review the combined results of the baselining 
# using all features and default parameters
results_df[['Classifier', 'Features', 'Parameters', 'Accuracy', 'Recall',
            'Precision', 'AUC', 'True_Positives', 'True_Negatives', 
            'False_Positives', 'False_Negatives', 'Cost(1000s)']].sort_values(by = 'Cost(1000s)')

,Classifier,Features,Parameters,Accuracy,Recall,Precision,AUC,True_Positives,True_Negatives,False_Positives,False_Negatives,Cost(1000s)
4,XGB,Full,Default,0.921643,0.921643,0.911690,0.973989,14288,22551,1384,1748,887.84
3,Random Forest,Full,Default,0.917565,0.917565,0.926834,0.971895,13833,22843,1092,2203,1112.42
2,Decision Tree,Full,Default,0.839384,0.839384,0.798522,0.833216,12861,20690,3245,3175,1619.95
1,Gaussian Naive-Bayes,Full,Default,0.689400,0.689400,0.639344,0.742411,8307,19249,4686,7729,3911.36
0,Logistic Regression,Full,Default,0.707238,0.707238,0.678119,0.765414,8250,20019,3916,7786,3932.16


---
# Dimensionality Reduction

### Feature Reduction (RFECV)

In the provided dataset we have 50 variables.  It is essential to understand and determine the importance of each variable in model contribution to enhance estimator results and improve performance. This is an important concept in machine learning and centered around simplicity and optimization.  Using the sklearn.feature_selection module that examines the classes, we can determine the features having the greatest impact and contribution to the selected models. 

Reference: https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15

---
#### Benefits of Dimensionality reduction
- Contributes to increasing the goodness of fit metric - **R2**
- Aligns with the principle of **Occam's razor or the law of parsimony** - simpler is better and more likely to be correct
- Improve an estimators accuracy scores
- Boost performance especially on high-dimensional datasets

We used **Recursive Feature Elimination with Cross-Validation (RFECV)** to restrict the features used in the model.
Recursive feature elimination (RFE) selects features by recursively considering smaller and smaller sets of features by an external estimator based on the assignment of weights against the features (e.g. the coefficients of a linear model).
- Train the estimator on the baseline or complete set of features
- Importance of each feature is determined by the coef_ attribute or a feature_importances_ attribute. 
- Least important features are trimmed from the original feature set.
- The process is recursively repeated on the trimmed dataset until the optimal features desired are reached.

---
#### Parameters for RFECV: RFECV performs RFE in a cross-validation loop to identify the optimal features.
- **estimator**: A supervised learning estimator with a fit method that provides information about feature importance.
    - Parameter selected: **RandomForestClassifier**
    
    
- **step** - If greater than or equal to 1, then step corresponds to the number of features to remove at each iteration.
    - Parameter selected: **4**
    
    
- **cv**: Determines the cross-validation splitting strategy. Integer value specifies the number of folds.
    - Parameter selected: **3**
    
    
- **scoring**: A scoring method to determine which score to maximize
    - Parameter selected: **recall**
---

# Joe - Please review the Feature Importance code as it would not run...errored out.  I have commented it out for now as I am running the entire notebook start to finish.

In [55]:
# Merge field names and feature importance to display them together
# fi = pd.DataFrame(RandomForestClassifier.feature_importances_, columns =['featimp'])
# featuresnames = pd.DataFrame(Business_Data_ohe_scaled.columns.values.tolist(), columns =['fields'])
# featimpdf = pd.merge(featuresnames, fi, left_index=True, right_index=True)
# featimpdf = featimpdf.sort_values(by='featimp', ascending=False)
# featimpdf


# plt.figure(figsize=(25,8))
# ax = sns.barplot(x=featimpdf.fields, y=featimpdf.featimp)
# ax.set_title('FEATURE IMPORTANCE')
# plt.xlabel("Feature")
# plt.ylabel("Importance")
# ax.set_xticklabels(ax.get_xticklabels(), rotation=65, horizontalalignment='right')

In [56]:
# Establish the parameters for RFECV using the Random Forest estimator and run the model
clf = RandomForestClassifier(n_estimators=100, random_state=rand_state)
selector = RFECV(estimator=clf, step = 4, cv=3, scoring='recall')

feature_fit = selector.fit(X_train, y_train)

print('Optimal count of features selected using RFECV:: ', feature_fit.n_features_)
print('Optimal features selected:: ', X_train.columns[selector.support_])

Optimal count of features selected using RFECV::  14
Optimal features selected::  Index(['x7', 'x12', 'x20', 'x23', 'x27', 'x28', 'x37', 'x38', 'x40', 'x41',
       'x42', 'x46', 'x48', 'x49'],
      dtype='object')


In [ ]:
# Plot features selected against the model score
plt.figure(figsize=(18,6))
plt.title('Recursive Feature CV score versus Feature count')
plt.xlabel("Features selected")
plt.ylabel("Recall")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.show()

In [ ]:
# Set the training and test sets using just the reduced features 
# identified from the RFECV operation conducted above
X_train_reduced_features = X_train[X_train.columns[selector.support_]]
X_test_reduced_features = X_test[X_test.columns[selector.support_]]

In [ ]:
# Review the reduced features for the training data
X_train_reduced_features.head(2)

In [ ]:
# Review the reduced features for the testing data
X_test_reduced_features.head(2)

---
## Training the models using the reduced features

Using default parameters to establish a baseline with reduced features before proceeding to the **parameter tuning phase** using a **Randomized Search** approach.

---
### Decision Tree Model

Decision trees are primarily leveraged to address both regression and classification cases. 
- **Classification tree models are used to predict a qualitative response**
- **Regression tree models are used to predict a quantitative response**

The algorithm constructs a tree using a training dataset where each node is an attribute and the branches are the corresponding values.

---

In [ ]:
# Establish the parameters for DECISION TREE and run the model
clf_DT_default = DecisionTreeClassifier(random_state=rand_state)
clf_DT_default.fit(X_train_reduced_features, y_train)
y_pred = clf_DT_default.predict(X_test_reduced_features)

y_pred_proba = clf_DT_default.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred)
results_df = results_df.append({'Classifier':'Decision Tree'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Default'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

---
### Random Forest Classifier

Random Forests are primarily based on Decision trees. 
- Are an ensemble of decision trees trained using the bagging methodology. 
- Leverages randomness when growing trees rather than just searching for the best feature when splitting a node.
- Seeks the best feature among a random subset of features. 
- Delivers an enhanced model by trading higher bias for lower variance. 

Random Forests are useful in gaining a good understanding of feature importance especially when performing feature selection.

---

In [ ]:
# Establish the parameters for RANDOM FOREST classifier and run the model
clf_RF_default = RandomForestClassifier(random_state=rand_state)
clf_RF_default.fit(X_train_reduced_features, y_train)
y_pred = clf_RF_default.predict(X_test_reduced_features)

y_pred_proba = clf_RF_default.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred)
results_df = results_df.append({'Classifier':'Random Forest'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Default'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

### XGBoost Classifier

XGBoost is an optimized distributed gradient boosting library designed for high efficiency, flexibility and portability. 
- Machine learning algorithms are deployed leveraging the Gradient Boosting framework. 
- It provides a parallel tree boosting approach (GBDT, GBM) which is fast and accurate for solving data science problems. 

In [ ]:
# Establish the parameters for XGBOOST Classifier and run the model
clf_XGB_default = XGBClassifier(random_state=rand_state)
clf_XGB_default.fit(X_train_reduced_features, y_train)
y_pred = clf_XGB_default.predict(X_test_reduced_features)

y_pred_proba = clf_XGB_default.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred)
results_df = results_df.append({'Classifier':'XGB'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Default'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

In [ ]:
# Review the combined results matrix
results_df

# RandomizedSearchCV to choose best parameters

In RandomizedSearchCV all hyperparameter values are not tested. Instead, a fixed number of hyperparameter settings are sampled from specified probability distributions.

#### Options for RandomizedSearchCV
- estimator - A object of that type is instantiated for each grid point.
    - Assigned as per algorithm being tuned
    
    
- param_distributions - Dictionary with parameters names (string) as keys and distributions or lists of parameters to try.
    - Assigned as per algorithm being tuned
    
    
- scoring - A single string to evaluate the predictions on the test set.
    - Parameter setting: **['precision' , 'recall' , 'accuracy']**
    
    
- cv - Determines the cross-validation splitting strategy.
    - Parameter setting: **5**
    
    
- refit - Refit an estimator using the best found parameters on the whole dataset.
    - Parameter setting: **recall**
    
    
- n_iterint - Number of parameter settings that are sampled. n_iter trades off runtime vs quality of the solution.
    - Parameter setting: **10**

In [1]:
# Initialize scores variable with metrics
scores = ['precision'
          , 'recall'
          , 'accuracy'
         ]
# Review the variable
print(scores)

['precision', 'recall', 'accuracy']


In [ ]:
# Define a custom function to capture the performance metrics
def report_perf(clf, n_top = 3, clf_name = "", verbose=True):
    if(verbose):
        for n in range(1, n_top+1):
            candidates = np.flatnonzero(clf.cv_results_['rank_test_recall'] == n)
            for candidate in candidates:
                print('Model with rank: {0}'.format(n))
                print('Mean validation score (Recall on Test): {0:.3f} (std: {1:.3f})'.format(
                    clf.cv_results_['mean_test_recall'][candidate], clf.cv_results_['std_test_recall'][candidate]))
                print('ParametersL {0}',format(clf.cv_results_['params'][candidate]))
    return{'Classifier': clf_name
           , 'Best_Parameters': str(clf.best_params_)
           , 'Best_Estimator': str(clf.best_estimator_)
           , 'Precision_Mean':clf.cv_results_['mean_test_precision'][clf.best_index_]
           , 'Precision_std':clf.cv_results_['std_test_precision'][clf.best_index_]
           , 'Recall_Mean':clf.cv_results_['mean_test_recall'][clf.best_index_]
           , 'Recall_std':clf.cv_results_['std_test_recall'][clf.best_index_]
          }

# Decision Tree

Tuning the parameters for the *Decision Tree Classifier*:
- max_depth: The maximum depth of the tree.
    - Parameter setting: **[10, 20, 30, None]**
    
    
- max_features: The number of features to consider when looking for the best split
    - Parameter setting: **['auto', 'sqrt']**
    
    
- criterion: The function to measure the quality of a split. Supported criteria are "gini" for the Gini impurity and "entropy" for the information gain.
    - Parameter setting: **['gini','entropy']**
    
    
- min_samples_leaf: The minimum number of samples required to be at a leaf node.  
    - Parameter setting: **[1, 2, 4]**
    
    
- min_samples_split: The minimum number of samples required to split an internal node
    - Parameter setting: **[2, 5, 10]**
    
    
- random_state - If int, random_state is the seed used by the random number generator
    - Parameter setting: **10**

In [ ]:
tuned_parameters_DT={'max_depth': [10, 20, 30, None]
                     , 'max_features': ['auto', 'sqrt']
                     , 'criterion':['gini','entropy']
                     , 'min_samples_leaf': [1, 2, 4]
                     , 'min_samples_split': [2, 5, 10]
                    }

clf_DT = DecisionTreeClassifier(random_state=rand_state)

random_search_DT = RandomizedSearchCV(estimator=clf_DT
                                      , param_distributions=tuned_parameters_DT
                                      , cv = 5
                                      , scoring = scores
                                      , refit ='recall'
                                      , n_iter=10
                                     )

random_search_DT.fit(X_train_reduced_features, y_train)

In [ ]:
DT_report = report_perf(random_search_DT, n_top = 3, clf_name='Decision_Tree')
DT_report

# Random Forest

Tuning the parameters for the *Random Forest Classifier:
- n_estimators: The number of trees in the forest.
    - Parameter setting: **[100, 200]**
    
    
- max_depth: The maximum depth of the tree.
    - Parameter setting: **[25, 50, None]**
    
    
- criterion: function to measure the quality of a split. Supported criteria are "gini" for Gini impurity and "entropy" for information gain.
    - Parameter setting: **['gini','entropy']**
    
    
- min_samples_leaf: The minimum number of samples required to be at a leaf node.
    - Parameter setting: **[1, 25, 50]**
    
    
- max_features: The number of features to consider when looking for the best split.
    - Parameter setting: **[.2, .3, 'auto']**
    
    
- min_samples_split: The minimum number of samples required to split an internal node.
    - Parameter setting: **[2, 4, 8, 10, 12]**

In [ ]:
tuned_parameters_RF={'n_estimators':[100, 200]
                     , 'max_depth':[25, 50, None]
                     , 'criterion':['gini','entropy']
                     , 'min_samples_leaf': [1, 25, 50]
                     , 'max_features':[.2, .3, 'auto']
                     , 'min_samples_split':[2, 4, 8, 10, 12]
                    }

clf_RF = RandomForestClassifier(random_state=rand_state)

random_search_RF = RandomizedSearchCV(estimator=clf_RF
                                      , param_distributions=tuned_parameters_RF
                                      , cv = 5
                                      , scoring = scores
                                      , refit = 'recall'
                                      , n_iter=10
                                     )

random_search_RF.fit(X_train_reduced_features, y_train)

In [ ]:
RF_report = report_perf(random_search_RF, n_top = 3, clf_name='Random_Forest')
RF_report

# XGBoost Classifier

Tuning the parameters for XGBoost:
- n_estimators: Number of gradient boosted trees. Equivalent to number of boosting rounds.
    - Parameter setting: **[100, 150, 1000]**
    
    
- learning_rate: Boosting learning rate (xgb’s “eta”)
    - Parameter setting: **[0.01, 0.6, None]**
    
    
- subsample: Subsample ratio of the training instances.
    - Parameter setting: **[0.3, 0.9, None]**
    
    
- max_depth: Maximum depth of a tree.
    - Parameter setting: **[3, 4, 5, 6, 7, 8, 9, None]**
    
    
- colsample_bytree: The subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed.
    - Parameter setting: **[0.5, 0.9, None]**
    
    
- min_child_weight: Minimum sum of instance weight (hessian) needed in a child.
    - Parameter setting: **[1, 2, 3, 4, None]**

In [ ]:
tuned_parameters_XGB={'n_estimators': [100, 150, 1000]
                      , 'learning_rate': [0.01, 0.6, None]
                      , 'subsample': [0.3, 0.9, None]
                      , 'max_depth': [3, 4, 5, 6, 7, 8, 9, None]
                      , 'colsample_bytree': [0.5, 0.9, None]
                      , 'min_child_weight': [1, 2, 3, 4, None]
                     }

clf_XGB = XGBClassifier(random_state=rand_state)

In [ ]:
random_search_XGB = RandomizedSearchCV(estimator=clf_XGB
                                      , param_distributions=tuned_parameters_XGB
                                      , cv = 5
                                      , scoring = scores
                                      , refit ='recall'
                                      , n_iter=10
                                     )

In [ ]:
random_search_XGB.fit(X_train_reduced_features, y_train)

In [ ]:
XGB_report = report_perf(random_search_XGB, n_top = 3, clf_name='XGBoost')

### Append random_state to the best parameters

We assign the best parameters from each of the Random Search to a variable and then add the random state parameter to it. These variables will be passed on to train the models with reduced features and best parameters.

In [ ]:
DT_best_params = random_search_DT.best_params_
DT_best_params['random_state'] = rand_state

RF_best_params = random_search_RF.best_params_
RF_best_params['random_state'] = rand_state

XGB_best_params = random_search_XGB.best_params_
XGB_best_params['random_state'] = rand_state

### Train the model with the best parameters.

### Decision Tree
- The best parameter (with random_state) from the previous step is chosen as classifier
- Confusion Matrix and Classification Reports are displayed 

In [ ]:
clf_DT_best = DecisionTreeClassifier(**DT_best_params)
clf_DT_best.fit(X_train_reduced_features, y_train)

y_pred_DT = clf_DT_best.predict(X_test_reduced_features)

y_pred_proba = clf_DT_best.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_DT)
results_df = results_df.append({'Classifier':'Decision Tree'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

### Random Forest
- The best estimator from the previous step is chosen as classifier
- Confusion Matrix and Classification Reports are displayed 

In [ ]:
clf_RF_best = RandomForestClassifier(**RF_best_params)
clf_RF_best.fit(X_train_reduced_features, y_train)

y_pred_RF = clf_RF_best.predict(X_test_reduced_features)

y_pred_proba = clf_RF_best.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_RF)
results_df = results_df.append({'Classifier':'Random Forest'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

### XGBoost
- The best estimator from the previous step is chosen as classifier
- Confusion Matrix and Classification Reports are displayed 

In [ ]:
clf_XGB_Best = XGBClassifier(**XGB_best_params)
clf_XGB_Best.fit(X_train_reduced_features, y_train)

y_pred_XGB = clf_XGB_Best.predict(X_test_reduced_features)

y_pred_proba = clf_XGB_Best.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_XGB)
results_df = results_df.append({'Classifier':'XGB'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

In [ ]:
results_df

# Ensemble Models

The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator.

Stacking, also called Super Learning or Stacked Regression, is a class of algorithms that involves training a second-level "metalearner" to find the optimal combination of the base learners. Unlike bagging and boosting, the goal in stacking is to ensemble strong, diverse sets of learners together.

![title](img/EnsembleModels3.png) 

References: https://www.kdnuggets.com/2019/01/ensemble-learning-5-main-approaches.html


### Ensemble Model 1 - Stacking

The first ensemble technique uses the vecstack package for stacking. It is compatible with the scikit-learn API to automate OOF computation, prediction and bagging using multiple models and stages.

![title](img/EnsembleModels.png) 

Reference: https://blogs.sas.com/content/subconsciousmusings/2017/05/18/stacked-ensemble-models-win-data-science-competitions/

Below, we use the Best Decision Tree and Random Forest classifiers in Stage 1. The Stage 1 output is passed on to Stage 2 which uses the XGBoost classifier with the best parameters. Finally, Logistic Regression is used as the Meta Classifier to predict the outcome and compare against the test dataset. 

In [ ]:
models_1 = [clf_DT_best
            , clf_RF_best]

# Get out-of-fold predictions from 1-level models
S_1_train, S_1_test = stacking(models_1
                               , X_train_reduced_features
                               , y_train
                               , X_test_reduced_features
                               , regression = True
                               , metric=recall_score
                               , verbose = 2)

In [ ]:
# Initialize 2-level models
models_2 = [clf_XGB_Best]

# Get out-of-fold predictions from 2-level models
S_2_train, S_2_test = stacking(models_2
                               , S_1_train
                               , y_train
                               , S_1_test
                               , regression = True
                               , metric=recall_score
                               , verbose = 2)

In [ ]:
# Initialize 3-level model
model = LogisticRegression(random_state=rand_state)
# Fit
model.fit(S_2_train, y_train)
# Get final prediction
y_pred_ensemble = model.predict(S_2_test)

In [ ]:
model.predict_proba(S_2_test)

In [ ]:
y_pred_proba = model.predict_proba(S_2_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_ensemble)
results_df = results_df.append({'Classifier':'Ensemble_1'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

In [ ]:
recall_score(y_test, y_pred_ensemble)

### Ensemble Model 2

The second ensemble technique uses the StackingCVClassifier from mlxtend package. 

![title](img/Ensemble_StackingCVClassifier.png) 

Reference: http://rasbt.github.io/mlxtend/user_guide/classifier/StackingCVClassifier/

In this model, we pass all our 3 best models (Decision Tree, Random Forest and XGBoost) together in 1 stage. Similar to the previous ensemble model, we use Logistic Regression as our Meta-Classifier.

In [ ]:
sclf = StackingCVClassifier(classifiers=[clf_DT_best, clf_RF_best, clf_XGB_Best]
                            , use_probas=True
                            , random_state= rand_state
                            , meta_classifier=LogisticRegression(random_state=rand_state))

In [ ]:
label = ['Decision Tree', 'Random Forest', 'XGBoost', 'Stacking Classifier']
clf_list = [clf_DT_best, clf_RF_best, clf_XGB_Best, sclf]
    
sclf.fit(X_train_reduced_features, y_train)
y_pred_sclf = sclf.predict(X_test_reduced_features)

In [ ]:
y_pred_proba = sclf.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_sclf)
results_df = results_df.append({'Classifier':'Ensemble_2'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

### Ensemble Model 3

Stack of estimators with a final classifier. Stacked generalization consists in stacking the output of individual estimator and use a classifier to compute the final prediction. Stacking allows to use the strength of each individual estimator by using their output as input of a final estimator.

![title](img/StackOfEstimatorsWithFinalClassifier.png)

Reference: https://towardsdatascience.com/stacking-classifiers-for-higher-predictive-performance-566f963e4840


In this step, we use multi-layer classsifier technique. In multi-layer classifier, we assign the final estimator to a stacking classifier.

In [ ]:
final_layer = StackingClassifier(estimators=[('rf', clf_RF_best)
                                            , ('xgb', clf_XGB_Best)]
                                , final_estimator=LogisticRegression(random_state=rand_state)
                               )

multi_layer_regressor = StackingClassifier(estimators=[('dt', clf_DT_best)]
                                          , final_estimator=final_layer
                                         )

multi_layer_regressor.fit(X_train_reduced_features, y_train)

y_pred_multi = multi_layer_regressor.predict(X_test_reduced_features)

In [ ]:
y_pred_proba = multi_layer_regressor.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_multi)
results_df = results_df.append({'Classifier':'Ensemble_3'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

### Ensemble Model 4

Ensemble Model 4 is similar to Ensemble Model 3. However, in this step, we use a single stacking layer with Decision Tree, Random Forest and XGBoost with the best parameters passed as ensemble estimators. 

![title](img/EnsembleModels2.png)

Reference: https://blogs.sas.com/content/subconsciousmusings/2017/05/18/stacked-ensemble-models-win-data-science-competitions/


SGD is chosen as the final estimator or meta-classifier as discussed in Ensemble Model 1 and 2.

In [ ]:
ensemble_estimators = [('rf', clf_RF_best)
                       , ('xgb', clf_XGB_Best)
                       , ('dt', clf_DT_best)
                      ]

clf_ensemble = StackingClassifier(estimators=ensemble_estimators
                                  , final_estimator=CalibratedClassifierCV(SGDClassifier(random_state=rand_state))
                                 )

clf_ensemble.fit(X_train_reduced_features, y_train)

In [ ]:
y_pred_ens = clf_ensemble.predict(X_test_reduced_features)

In [ ]:
y_pred_proba = clf_ensemble.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_ens)
results_df = results_df.append({'Classifier':'Ensemble_4'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

# Ensemble Model 5

Ensemble model 5 is the same as Ensemble Model 4. The only difference is that we are using SMV (SVC) as the final estimator here instead of SGD.

In [ ]:
ensemble_estimators_5 = [('rf', clf_RF_best)
                       , ('xgb', clf_XGB_Best)
                       , ('dt', clf_DT_best)
                      ]

clf_ensemble_5 = StackingClassifier(estimators=ensemble_estimators_5
                                    , final_estimator=CalibratedClassifierCV(SVC(gamma = 0.5, random_state=rand_state))
                                 )

clf_ensemble_5.fit(X_train_reduced_features, y_train)

In [ ]:
y_pred_ens_5 = clf_ensemble_5.predict(X_test_reduced_features)

In [ ]:
y_pred_proba = clf_ensemble_5.predict_proba(X_test_reduced_features)[:, 1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

(FP, FN, TP, TN, Cost, Accuracy, Recall, Precision) = confusion_mat(y_test, y_pred_ens_5)
results_df = results_df.append({'Classifier':'Ensemble_5'
                                , 'Features':'Reduced'
                                , 'Parameters': 'Best'
                                , 'Accuracy': Accuracy
                                , 'Recall': Recall
                                , 'Precision': Precision
                                , 'FPR': fpr
                                , 'TPR': tpr
                                , 'AUC': auc
                                , 'True_Positives': TP
                                , 'True_Negatives': TN
                                , 'False_Positives': FP
                                , 'False_Negatives': FN
                                , 'Cost': Cost
                               }
                               , ignore_index=True
                              )

In [ ]:
results_df[['Classifier', 'Features', 'Parameters', 'Accuracy', 'Recall',
            'Precision', 'AUC', 'True_Positives', 'True_Negatives', 
            'False_Positives', 'False_Negatives', 'Cost']].sort_values(by = 'Cost')

In [ ]:
roc_plot_df = results_df[['Classifier', 'Features', 'Parameters', 
                          'AUC', 'FPR', 'TPR', 'Cost']].sort_values(by = 'Cost').head(10)

In [ ]:
roc_plot_df['Clf'] = roc_plot_df[['Classifier', 'Features', 'Parameters']].apply(lambda x: '_'.join(x), axis=1)

roc_plot_df

In [ ]:
roc_plot_df.set_index('Clf', inplace=True)

In [ ]:
fig = plt.figure(figsize=(8,6))

for i in roc_plot_df.index:
    plt.plot(roc_plot_df.loc[i]['FPR'], 
             roc_plot_df.loc[i]['TPR'], 
             label="{}, AUC={:.3f}".format(i, roc_plot_df.loc[i]['AUC']))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()

# Conclusion


# References

Code adapted from Capstone Project work by Kevin Kendonsa, Vivek Viswanathan, and Maureen Stolberg: "TITLE HERE"

- https://jamesrledoux.com/code/imputation
- https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
- https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas
- https://stackoverflow.com/questions/60393024/rfecv-for-classification-giving-keyerror-weight
- https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html
- https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python
- https://towardsdatascience.com/automate-stacking-in-python-fc3e7834772e
- https://scikit-learn.org/stable/modules/ensemble.html